# Competitive Pokemon Analysis

## Goal

## Results

## Updates

In [2]:
import pandas as pd
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

# Get Pokemon Stats CSV File
Pkmn_raw = pd.read_csv('Pokemon.csv')

In [3]:
# CLEAN POKEMON NAMES
# Assuming it's fine to keep as _raw
#Pandas Multiple Replace
#https://stackoverflow.com/questions/36072626/pandas-replace-multiple-values-at-once

Name_Replacements = {
    'Name':{
        # Remove duplicate names found in Mega, Primal, and Hoopa
        # Regex: ?= -> Look ahead Assertion
        # [Regex -> English] Replace everything before "Target Word" with empty space 
        # Pkmn_raw['Name'] = Pkmn_raw['Name'].str.replace(".*(?=Mega)", "")
        r'.*(?=Mega|Primal|Hoopa)': '',
        # Add Space between  Capitals to account in Forme, Cloake, Mode, Size
        # Regex:
        # [Regex -> English] Find two consecutive groups of (Words) and (Capital Letter) or ("50%") and Add a space between the two groups
        r'(\w)([A-Z])|(?=50%)': r'\1 \2'
    }
}

Pkmn_raw.replace(Name_Replacements, regex=True, inplace=True)

In [4]:
# CLEAN POKEMON GEN
# Issue: Mega Evolutions and Primal Reversions are Gen 6, but are appended to the respective Pokemon in the Pokedex

Pkmn_raw['Gen_Normalized'] = Pkmn_raw['Generation']
Pkmn_raw.loc[Pkmn_raw['Name'].str.contains('Mega|Primal'), 'Gen_Normalized'] = 6

#Pkmn_raw['Generation Normalized'] = Pkmn_raw['Generation'].replace(Gen_Replacements,regex=True)

# Make Generation Normalized = 6 for Mega and Primal Transform


In [ ]:
# Box Plot of Base Stats per Generation (I-VI)
trace_all = go.Box(
                y = Pkmn_raw['Total'],
                name = 'All Gen', 
                boxpoints = 'all', 
                text = Pkmn_raw['Name'],
                pointpos = -2,
                jitter = 0.6,
                boxmean = 'sd',
                )

data = [trace_all]
for i in Pkmn_raw['Gen_Normalized'].sort_values().unique():
    trace = go.Box(
        y = Pkmn_raw.loc[Pkmn_raw['Gen_Normalized'] == i, 'Total'],
        name = "Gen " + str(i),
        boxpoints = 'all',
        text = Pkmn_raw.loc[Pkmn_raw['Gen_Normalized'] == i,'Name'],
        jitter = 0.6,
        boxmean = 'sd',
        pointpos = -2,
    )    
    data.append(trace)

layout = go.Layout(
    title = 'Total Stats Per Generation (I-VI)',
    autosize = False,
    yaxis = dict(range = [100, 900]),
    height = 2000,
    width = 4000,
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig)



In [ ]:
# POLAR SCATTER [CURRENTLY NOT IN USE!]
# Starter comparison
# 0 - 200
#         HP
#  ATTACK   SPECIAL ATTACK
#  DEFENSE  SPECIAL DEFENSE
#        SPEED
# HP ATTACK DEFENSE SP ATTACK SP DEF SPEED
# Set up plot [Check]
# Reorient Categories [Check]
# Value depending on pokemon name and column [Check]


# def PkmnPolarStats(Pkmn_Comparison):
#     data = []
#     for Mon in Pkmn_Comparison:
#         Stats_Polar = go.Scatterpolar(
#             name = Mon,
#             r = [
#                  Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'HP'], 
#                  Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Sp. Atk'], 
#                  Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Sp. Def'], 
#                  Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Speed'], 
#                  Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Defense'], 
#                  Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'Attack'], 
#                  Pkmn_raw.loc[Pkmn_raw['Name'] == Mon,'HP']
#                 ],
#             theta = ['HP','Sp. Atk','Sp. Def','Speed','Defense','Attack','HP'],
#             fill = 'toself',
#             hoveron = 'points',
#         )
#         data.append(Stats_Polar)
#     return data

# testdata1 = PkmnPolarStats(['Bulbasaur','Ivysaur','Squirtle','Wartortle','Charmander','Charmeleon'])
# testdata2 = PkmnPolarStats(['Squirtle','Wartortle'])
# testdata3 = PkmnPolarStats(['Charmander','Charmeleon'])

# layout = go.Layout(   
#     polar = dict(
#         radialaxis = dict(
#             range = [0,200],
#         ),
#         angularaxis = dict(
#             rotation = 90,
#         ),
#     ),
#     showlegend = True,
#     title = 'TestTitle',
# )


# fig = go.Figure(data=testdata1,layout=layout)
# py.iplot(fig, filename='polar-category')

In [ ]:
Pkmn_raw

In [5]:
# Creating Starter Dataframes from Raw Dataframe
Pkmn_Starter_Grass = ['Bulbasaur', 'Chikorita', 'Treecko', 'Turtwig', 'Snivy', 'Chespin']
Grass_Starters = Pkmn_raw.loc[Pkmn_raw['Name'].isin(Pkmn_Starter_Grass)].drop(columns=['Generation','Legendary','Gen_Normalized'])

Pkmn_Starter_Fire = ['Charmander', 'Cyndaquil', 'Torchic', 'Chimchar', 'Tepig', 'Fennekin']
Fire_Starters = Pkmn_raw.loc[Pkmn_raw['Name'].isin(Pkmn_Starter_Fire)].drop(columns=['Generation','Legendary','Gen_Normalized'])

Pkmn_Starter_Water = ['Squirtle', 'Totodile', 'Mudkip', 'Piplup', 'Oshawott', 'Froakie']
Water_Starters = Pkmn_raw.loc[Pkmn_raw['Name'].isin(Pkmn_Starter_Water)].drop(columns=['Generation','Legendary','Gen_Normalized'])


In [ ]:
# Starter Comparison TEST
def ComparisonBoxPlot(Mon_df, Stats_list):
    data = []
    for Stat in Stats_list:
        Box_Trace= go.Box(
            y=Mon_df[Stat],
            name = Stat,
            boxpoints='all',
            text=Mon_df['Name'],
        ) 
    
        data.append(Box_Trace)
    return data

Pkmn_Stats = ['HP','Attack','Defense','Sp. Atk', 'Sp. Def', 'Speed']
Test_Grass = ComparisonBoxPlot(Grass_Starters, Pkmn_Stats)

layout = go.Layout(
    title='Test',
    autosize=True,
)
fig = go.Figure(data=Test_Grass, layout=layout)
py.iplot(fig)




In [8]:
# Starter Comparison Final
Stats_List=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
x = np.repeat(Stats_List,Pkmn_raw['Generation'].max()) #6 for All 6 generations of (grass/fire/water) type pokemen

trace0 = go.Box(
    y=pd.concat([Grass_Starters['HP'],
                 Grass_Starters['Attack'],
                 Grass_Starters['Defense'],
                 Grass_Starters['Sp. Atk'],
                 Grass_Starters['Sp. Def'],
                 Grass_Starters['Speed']
                ]),
    x=x,
    name='Grass',
    marker=dict(
        color='#78C850'
    ),
    boxpoints='all',
    text=pd.concat([Grass_Starters['Name']] * Pkmn_raw['Generation'].max())
)
trace1 = go.Box(
    y=pd.concat([Fire_Starters['HP'],
                 Fire_Starters['Attack'],
                 Fire_Starters['Defense'],
                 Fire_Starters['Sp. Atk'],
                 Fire_Starters['Sp. Def'],
                 Fire_Starters['Speed']
                ]),
    x=x,
    name='Fire',
    marker=dict(
        color='#F08030'
    ),
    boxpoints='all',
    text = pd.concat([Fire_Starters['Name']] * Pkmn_raw['Generation'].max())
)
trace2 = go.Box(
    y=pd.concat([Water_Starters['HP'],
                 Water_Starters['Attack'],
                 Water_Starters['Defense'],
                 Water_Starters['Sp. Atk'],
                 Water_Starters['Sp. Def'],
                 Water_Starters['Speed']
                ]),
    x=x,
    name='Water',
    marker=dict(
        color='#6890F0'
    ),
    boxpoints='all',
    text=pd.concat([Water_Starters['Name']] * Pkmn_raw['Generation'].max())
)
layout = go.Layout(
    title = 'Pokemon Starters',
    yaxis=dict(
        zeroline=False
    ),
    boxmode='group'
)

fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)
py.iplot(fig)




$e^{i\pi} + 1 = 0$


## $$ Damage = \Bigg(\frac{2 \times Level}{5}\Big)+2 $$


# $$ Damage = \Bigg( \frac{\big(\frac{2 \times Level}{5} +2 \big) \times Power \times A/D}{50} +2 \Bigg) \times Modifier$$

# $$ Modifier = Targets \times Weather \times Critical \times random \times STAB \times Type \times Burn \times Other$$